In [1]:
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader,ConcatDataset
import math
from torch import nn
from lib.models import RegNet
from torch.nn.functional import relu
import matplotlib.pyplot as plt
import torch
from tqdm import tqdm
import numpy as np
import random

torch.manual_seed(0)
np.random.seed(0)
random.seed(0)

CONFIG = {
    'WINDOW_SIZE':5000,
    'BATCH_SIZE':1024,
    'LEARNING_RATE':3e-3,
    'DEVICE':'cuda',
    'SEQUENCE_LENGTH':7
}

In [4]:
from lib.models import Dumbledorev2
from lib.datasets import EpochedDataset,SequencedDatasetv2
from lib.ekyn import get_ekyn_ids,load_ekyn_pt

train_idx,test_idx = train_test_split(get_ekyn_ids(),test_size=.25,random_state=0)
trainloader = DataLoader(ConcatDataset([SequencedDatasetv2(idx=idx,condition=condition,sequence_length=CONFIG['SEQUENCE_LENGTH'],batch_size=CONFIG['BATCH_SIZE']) for idx in train_idx for condition in ['Vehicle','PF']]),batch_size=1,shuffle=True)
devloader = DataLoader(ConcatDataset([SequencedDatasetv2(idx=idx,condition=condition,sequence_length=CONFIG['SEQUENCE_LENGTH'],batch_size=CONFIG['BATCH_SIZE']) for idx in test_idx for condition in ['Vehicle','PF']]),batch_size=1,shuffle=True)
model = Dumbledorev2(encoder_path=f'projects/6',sequence_length=CONFIG['SEQUENCE_LENGTH'],hidden_dim=128,layers=1,embedding=True,batch_size=CONFIG['BATCH_SIZE'])
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(),lr=CONFIG['LEARNING_RATE'])
model.to(CONFIG['DEVICE']);

Model is freezing encoder


/home/andrew/.local/lib/python3.10/site-packages/torch/nn/modules/rnn.py:67: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [5]:
lossi = []
trainlossi = []
trainf1 = []
devlossi = []
devf1 = []
model.train()
from lib.utils import training_loop,development_loop
for i in tqdm(range(1000)):
    loss,f1 = training_loop(model=model,trainloader=trainloader,criterion=criterion,optimizer=optimizer,device=CONFIG['DEVICE'])
    trainlossi.append(loss)
    trainf1.append(f1)

    loss,f1 = development_loop(model=model,devloader=devloader,criterion=criterion,device=CONFIG['DEVICE'])
    devlossi.append(loss)
    devf1.append(f1)

    fig,ax = plt.subplots(nrows=1,ncols=2,figsize=(20,4))
    ax[0].plot(trainlossi)
    ax[0].plot(devlossi)
    ax[1].plot(trainf1)
    ax[1].plot(devf1)
    plt.savefig('loss.jpg')
    plt.close()

  1%|          | 8/1000 [00:51<1:46:34,  6.45s/it]

In [ ]:
from lib.utils import evaluate
evaluate(devloader,model,criterion,CONFIG['DEVICE'])